##### Reference links
> https://medium.com/dunder-data/create-a-bar-chart-race-animation-in-python-with-matplotlib-477ed1590096
<br>
> https://medium.com/@suryadayn/error-requested-moviewriter-ffmpeg-not-available-easy-fix-9d1890a487d3
<br>
##### Download ffmpeg file
> https://www.gyan.dev/ffmpeg/builds/packages/ffmpeg-4.3.1-2020-10-28-full_build.7z

In [1]:
import pandas as pd
from datetime import datetime, date
from matplotlib.ticker import StrMethodFormatter
import matplotlib.pyplot as plt
import seaborn as sb
from matplotlib.animation import FuncAnimation, PillowWriter
from IPython.display import HTML

In [2]:
df = pd.read_csv('data_preprocessing/general_nation_province_year.csv', index_col=0)
df.count()

Ma_Tinh         528
So_Diem_Chay    528
Nam             528
dtype: int64

In [3]:
df.head()

,Ma_Tinh,So_Diem_Chay,Nam
0,62,1460,2011
1,64,1082,2011
2,66,926,2011
3,40,612,2011
4,11,578,2011


In [4]:
df_province = pd.read_csv('data_preprocessing/Tinh_Forest.csv', index_col=0)
print(df_province.shape)
df_province.head(5)

(63, 2)


,Code,Name
0,89,An Giang
1,24,Bắc Giang
2,6,Bắc Kạn
3,95,Bạc Liêu
4,27,Bắc Ninh


In [5]:
dict_province = {}
for index in range(df_province.shape[0]):
    row = df_province.iloc[index]
    dict_province[row.values[0]] = row.values[1]

In [6]:
df_pivot = df.pivot(index='Nam', columns='Ma_Tinh', values='So_Diem_Chay')
print(df_pivot.shape)
columns = df_pivot.columns
columns = reversed(df_pivot.columns)
df_pivot = df_pivot[columns]
df_pivot.head(5)

(10, 63)


Ma_Tinh,96,95,94,93,92,91,89,87,86,84,...,15,14,12,11,10,8,6,4,2,1
Nam,,,,,,,,,,,,,,,,,,,,,
2011,NaN,NaN,NaN,1.0,NaN,13.0,6.0,2.0,NaN,NaN,...,42.0,326.0,227.0,578.0,10.0,49.0,71.0,35.0,34.0,NaN
2012,4.0,NaN,NaN,NaN,NaN,35.0,1.0,4.0,NaN,NaN,...,91.0,1055.0,1102.0,1567.0,82.0,87.0,148.0,81.0,72.0,NaN
2013,14.0,NaN,NaN,NaN,NaN,13.0,NaN,5.0,NaN,NaN,...,83.0,500.0,394.0,996.0,30.0,72.0,101.0,73.0,46.0,1.0
2014,3.0,NaN,1.0,2.0,NaN,15.0,9.0,NaN,NaN,NaN,...,122.0,974.0,1078.0,1706.0,80.0,71.0,59.0,66.0,50.0,2.0
2015,27.0,NaN,NaN,1.0,NaN,71.0,12.0,7.0,NaN,NaN,...,117.0,712.0,372.0,1013.0,61.0,148.0,134.0,115.0,119.0,1.0


In [7]:
df_pivot = df_pivot.rename(columns=dict_province)
df_pivot.head(5)

Ma_Tinh,Cà Mau,Bạc Liêu,Sóc Trăng,Hậu Giang,Cần Thơ,Kiên Giang,An Giang,Đồng Tháp,Vĩnh Long,Trà Vinh,...,Yên Bái,Sơn La,Lai Châu,Điện Biên,Lào Cai,Tuyên Quang,Bắc Kạn,Cao Bằng,Hà Giang,Hà Nội
Nam,,,,,,,,,,,,,,,,,,,,,
2011,NaN,NaN,NaN,1.0,NaN,13.0,6.0,2.0,NaN,NaN,...,42.0,326.0,227.0,578.0,10.0,49.0,71.0,35.0,34.0,NaN
2012,4.0,NaN,NaN,NaN,NaN,35.0,1.0,4.0,NaN,NaN,...,91.0,1055.0,1102.0,1567.0,82.0,87.0,148.0,81.0,72.0,NaN
2013,14.0,NaN,NaN,NaN,NaN,13.0,NaN,5.0,NaN,NaN,...,83.0,500.0,394.0,996.0,30.0,72.0,101.0,73.0,46.0,1.0
2014,3.0,NaN,1.0,2.0,NaN,15.0,9.0,NaN,NaN,NaN,...,122.0,974.0,1078.0,1706.0,80.0,71.0,59.0,66.0,50.0,2.0
2015,27.0,NaN,NaN,1.0,NaN,71.0,12.0,7.0,NaN,NaN,...,117.0,712.0,372.0,1013.0,61.0,148.0,134.0,115.0,119.0,1.0


In [8]:
def prepare_data(df, steps=5):
    df = df.reset_index()
    df.index = df.index * steps
    last_idx = df.index[-1] + 1
    df_expanded = df.reindex(range(last_idx))
    df_expanded['Nam'] = df_expanded['Nam'].fillna(method='ffill')
    df_expanded = df_expanded.set_index('Nam')
    df_expanded = df_expanded.interpolate()
    return df_expanded

df_expanded = prepare_data(df_pivot, 10)
print(df_expanded.shape)
df_expanded.head()

(91, 63)


Ma_Tinh,Cà Mau,Bạc Liêu,Sóc Trăng,Hậu Giang,Cần Thơ,Kiên Giang,An Giang,Đồng Tháp,Vĩnh Long,Trà Vinh,...,Yên Bái,Sơn La,Lai Châu,Điện Biên,Lào Cai,Tuyên Quang,Bắc Kạn,Cao Bằng,Hà Giang,Hà Nội
Nam,,,,,,,,,,,,,,,,,,,,,
2011.0,NaN,NaN,NaN,1.000000,NaN,13.0,6.0,2.0,NaN,NaN,...,42.0,326.0,227.0,578.0,10.0,49.0,71.0,35.0,34.0,NaN
2011.0,NaN,NaN,NaN,1.033333,NaN,15.2,5.5,2.2,NaN,NaN,...,46.9,398.9,314.5,676.9,17.2,52.8,78.7,39.6,37.8,NaN
2011.0,NaN,NaN,NaN,1.066667,NaN,17.4,5.0,2.4,NaN,NaN,...,51.8,471.8,402.0,775.8,24.4,56.6,86.4,44.2,41.6,NaN
2011.0,NaN,NaN,NaN,1.100000,NaN,19.6,4.5,2.6,NaN,NaN,...,56.7,544.7,489.5,874.7,31.6,60.4,94.1,48.8,45.4,NaN
2011.0,NaN,NaN,NaN,1.133333,NaN,21.8,4.0,2.8,NaN,NaN,...,61.6,617.6,577.0,973.6,38.8,64.2,101.8,53.4,49.2,NaN


In [9]:
NUM_COLORS = len(df_expanded.columns)
cm = plt.get_cmap('nipy_spectral')
colors = [cm(1.*i/NUM_COLORS) for i in range(NUM_COLORS)]
labels = df_expanded.columns

In [10]:
def nice_axes(ax):
    ax.set_facecolor('.8')
    ax.tick_params(labelsize=8, length=0)
    ax.grid(True, axis='x', color='white')
    ax.set_axisbelow(True)
    [spine.set_visible(False) for spine in ax.spines.values()]

In [18]:
def init():
    ax.clear()
    nice_axes(ax)
    ax.set_ylim(.2, 10)
    # ax.set_xticks([1000, 2000, 5000, 10000, 20000, 50000])

def update(i):
    for bar in ax.containers:
        bar.remove()
    row = df_expanded.iloc[i]
    y = row.index
    width = row.values
    barh = ax.barh(y=y, width=width, color=colors, tick_label=labels)
    date_str = date(int(df_expanded.index[i]), 1, 1).strftime('%Y')
    ax.set_title(f'Fire Forest - {date_str}', fontsize = 16, fontweight ="bold")
    return barh
    
fig = plt.Figure(figsize=(5, 10), dpi=120)
ax = fig.add_axes([0.2, 0.1, 0.75, 0.85])  # [left, bottom, width, height]
# ax = fig.add_subplot()
anim = FuncAnimation(
    fig=fig, 
    func=update, 
    init_func=init,
    frames=df_expanded.shape[0],
    interval=1000
)

In [19]:
html = anim.to_html5_video()
HTML(html)

In [13]:
anim.save('media/fire_forest.mp4')

In [21]:
from IPython.display import Video
Video("media/fire_forest.mp4")